# Практика на паре.

In [1]:
import numpy as np
import scipy.stats as ss
import pandas as pd
import matplotlib
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
%matplotlib inline

In [2]:
fin = open('data.txt', 'r')
games = [[int(t) for t in k.split()] for k in fin.readlines()]

In [3]:
len_games = sum([len(t) for t in games])

p_win = sum([sum(t) for t in games]) / len_games
p_go = len(games) / len_games
print(p_win, p_go)

0.48611867174741424 0.5443658138268916


In [7]:
stay = np.log(1-p_go)
leave = np.log(p_go)
win = np.log(p_win)
lose = np.log(1 - p_win)

def like_hood(a):
    pw, pg = a
    stay = np.log(1-pg)
    leave = np.log(pg)
    win = np.log(pw)
    lose = np.log(1 - pw)
    ret = 0
    for game in games:
        ret -= win * sum(game)
        ret -= lose * (len(game) - sum(game))
        ret -= leave
        ret -= stay * (len(game) - 1)
    return ret



In [9]:
from scipy.optimize import minimize
res = minimize(like_hood, x0=[0.5, 0.5], 
               bounds=[[np.power(10.0, -5), 1 - np.power(10.0, -5)], [np.power(10.0, -5), 1 - np.power(10.0, -5)]])
print(res)

      fun: 2538.6735555345367
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00318323,  0.00268301])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 4
     njev: 8
   status: 0
  success: True
        x: array([0.48611865, 0.54436567])


In [28]:
def new_model(a):
    pw, pg = a
    stay = np.log(1-pg)
    leave = np.log(pg)
    win = np.log(pw)
    lose = np.log(1 - pw)
    ret = 0
    for game in games:
        cntw = 0
        cnt = 0
        for t in game:
            cntw += t
            cnt += 1
            if cnt < len(game): ret -= np.log(1 - pg ** ((cntw + 1) / (cnt + 1)))
        ret -= win * sum(game)
        ret -= lose * (len(game) - sum(game))
        ret -= leave * ((cntw + 1) / (cnt + 1))
    return ret

In [29]:
res1 = minimize(new_model, x0=[0.5, 0.5], 
               bounds=[[1e-5, 1 - 1e-5], [1e-5, 1 - 1e-5]])
print(res1)

      fun: 2451.845983465085
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00172804, -0.009004  ])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 21
      nit: 4
     njev: 7
   status: 0
  success: True
        x: array([0.48611849, 0.41892264])


In [34]:
print(max([len(t) for t in games]))

8


In [46]:
def new_model2(a):
    pw, pg0, pg1 = a
    win = np.log(pw)
    lose = np.log(1 - pw)
    ret = 0
    for game in games:
        last = 0
        prel = 0
        for t in game:
            if t == 1: ret -= np.log(1-pg1)
            if t == 0: ret -= np.log(1 - pg0)
            prel, last = last, t
        ret -= win * sum(game)
        ret -= lose * (len(game) - sum(game))
        if last: ret -= np.log(pg1)
        if last == 0: ret -= np.log(pg0)
    return ret

In [47]:
res2 = minimize(new_model2, x0=[0.5, 0.5, 0.5], 
               bounds=[[1e-5, 1 - 1e-5], [1e-5, 1 - 1e-5], [1e-5, 1 - 1e-5]])
print(res2)

      fun: 3078.896120821343
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00682121,  0.01000444, -0.00427463])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 6
     njev: 9
   status: 0
  success: True
        x: array([0.48611782, 0.41692536, 0.26683095])
